In [5]:
# Импорт библиотек
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, split, regexp_replace, year, count, row_number
from pyspark.sql.window import Window
from pyspark.sql.types import StructType, StructField, StringType
import xml.etree.ElementTree as ET
import time

In [6]:
# Создаем Spark сессию
spark = SparkSession.builder \
    .appName("Top 10 Programming Languages Analysis") \
    .getOrCreate()

In [7]:
# Определение путей к файлам
posts_file = "posts_sample.xml"  # Используем сокращенную выборку
languages_file = "programming-languages.csv"
year_range = [str(year) for year in range(2010, 2021)]
num_top_languages = 10

In [8]:
# Засекаем начало выполнения
start_exec_time = time.time()

# Загрузка данных в RDD
raw_data_rdd = spark.read.text(posts_file).rdd.map(lambda row: row[0])

# Индексирование строк
indexed_rdd = raw_data_rdd.zipWithIndex()  # Добавление индекса к каждой строке
total_lines = indexed_rdd.count() # Подсчитываем общее количество строк
# Фильтрация строк: удаляем 1-ю, 2-ю и последнюю строку
filtered_rdd = indexed_rdd.filter(lambda entry: 1 < entry[1] < total_lines - 1).map(lambda entry: entry[0])

# Разбор XML-данных
def extract_data(xml_line):
    try:
      # Преобразуем строку в XML-элемент и извлекаем атрибуты
        element = ET.fromstring(xml_line)
        return (
            element.attrib.get("Id", ""),
            element.attrib.get("CreationDate", ""),
            element.attrib.get("Tags", "")
        )
    except:
        return None

parsed_rdd = filtered_rdd.map(extract_data).filter(lambda x: x is not None)

# Определение схемы данных для DataFrame
post_schema = StructType([
    StructField("Id", StringType(), True),
    StructField("CreationDate", StringType(), True),
    StructField("Tags", StringType(), True)
])

# Преобразование в DataFrame
df_posts = spark.createDataFrame(parsed_rdd, post_schema)

# Обработка полей: создание столбца "Year" и очистка тегов
df_posts = df_posts.withColumn("Year", year(col("CreationDate")))
df_posts = df_posts.withColumn("Tags", regexp_replace(col("Tags"), "[<>]", ""))
df_posts = df_posts.withColumn("Tags", split(col("Tags"), " "))

# Теги в отдельные строки
df_exploded = df_posts.select("Year", explode(col("Tags")).alias("Language"))

# Загрузка списка языков программирования
df_languages = spark.read.option("header", "true").csv(languages_file)
language_list = [row[0].lower() for row in df_languages.collect()]

# Фильтрация данных: выбираем только записи с нужными годами и языками
df_filtered = df_exploded.filter((col("Year").isin(year_range)) & (col("Language").isin(language_list))).cache()

# Подсчет количества упоминаний языков по годам
df_language_counts = df_filtered.groupBy("Year", "Language").agg(count("Language").alias("Count"))

# Выбор топ-N популярных языков
rank_window = Window.partitionBy("Year").orderBy(col("Count").desc()) # Окно для сортировки по убыванию частоты
df_top_languages = df_language_counts.withColumn("Rank", row_number().over(rank_window)).filter(col("Rank") <= num_top_languages)

# Сохранение результатов в формат Parquet
df_top_languages.write.mode("overwrite").parquet("top_languages_per_year.parquet")

In [9]:
# Чтение Parquet-файла
report_df = spark.read.parquet("top_languages_per_year.parquet")

# Показ содержимого DataFrame
report_df.show(n=report_df.count())

# Засекаем время выполнения
end_exec_time = time.time()
print(f"Total execution time: {end_exec_time - start_exec_time:.2f} seconds")

+----+-----------+-----+----+
|Year|   Language|Count|Rank|
+----+-----------+-----+----+
|2010|        php|    7|   1|
|2010|     python|    4|   2|
|2010|          c|    3|   3|
|2010|       java|    2|   4|
|2010|         go|    1|   5|
|2010|        ksh|    1|   6|
|2011|       java|    7|   1|
|2011|        php|    6|   2|
|2011|     python|    5|   3|
|2011|          c|    5|   4|
|2011| javascript|    4|   5|
|2011|objective-c|    1|   6|
|2011|       ruby|    1|   7|
|2011|       cuda|    1|   8|
|2011|    haskell|    1|   9|
|2011|     delphi|    1|  10|
|2012|        php|   17|   1|
|2012| javascript|    7|   2|
|2012|       java|    5|   3|
|2012|          c|    5|   4|
|2012|     python|    4|   5|
|2012|      scala|    3|   6|
|2012|       ruby|    3|   7|
|2012|objective-c|    2|   8|
|2012|          r|    2|   9|
|2012|       bash|    1|  10|
|2013| javascript|   15|   1|
|2013|        php|   13|   2|
|2013|       java|   10|   3|
|2013|          r|    8|   4|
|2013|    

In [10]:
# Завершаем сессию
spark.stop()